In [1]:
import numpy as np
import math
from src.texas_holdem_simp import Texas_Holdem_Simp

env = Texas_Holdem_Simp()
env.plot(env.enum(('J♥', 'J♠', 'Q♥', 'Q♠', 3,0,0)))

----(1)-----
|P2(5): J♠ |
|(2): Q♥-**|
|P1(3): J♥ |
------------


In [2]:

def lrtdp(max_iter=1000, epsilon=.1, gamma=.9):
    v = np.zeros(env.S.size)
    H = np.zeros(env.S.size)
    solved = [False for _ in env.S]

    curr_iter = 0

    bell = lambda s:  max([sum([env.T(s,a,s_)*(env.R(s,a,s_) + gamma*v[s_]) for s_ in env.S]) for a in env.A])
    greedy = lambda s: np.argmax([sum([env.T(s,a,s_)*(env.R(s,a,s_) + gamma*v[s_]) for s_ in env.S]) for a in env.A])

    def check_solved(s):
        rv = True
        opened, closed = [], []
        if not solved[s]:
            opened.append(s)
        while len(opened)>0:
            s = opened.pop()
            closed.append(s)
            if abs(bell(s)-v[s]) > epsilon:
                rv = False
                continue
            a = greedy(s)
            for s_ in env.S:
                if env.T(s,a,s_)>0:
                    if not solved[s_] and not(s_ in opened+closed):
                        opened.append(s_)
        if rv:
            for s_ in closed:
                if not solved[s_]:
                    solved[s_] = True
        else:
            for s_ in closed:
                v[s_] = bell(s_)
        return rv

    while not(all([solved[s0] for s0 in env.S0])) and curr_iter<max_iter:
        curr_iter += 1
        visited = []
        s = np.random.choice(env.S0)
        while not solved[s]:
            visited.append(s)
            if s in env.G: break
            a = greedy(s)
            v[s] = bell(s)
            s = np.random.choice(env.S, p=[env.T(s,a,s_) for s_ in env.S])
        while len(visited) > 0:
            s = visited.pop()
            if not check_solved(s): break

    # env.plot(list(np.round(v, 2)))
    # return [greedy(s) for s in env.S]
    return v
        

v = lrtdp(epsilon=.001, gamma=.99999)
# pi = lrtdp(epsilon=.001, gamma=.99999)
# sum(pi)

In [16]:
v0 = v[env.S0]
np.sum(v0)
# pi = np.argmax(q, axis=1)
len(v0[(v0>0)])/len(v0)
max(v0)
# np.prod(v0)

3.759646614599145

In [3]:
## 698m 43.3s
# with open("lrtdp.txt", "a") as f:
#     for item in pi:
#         f.write(f'{item}\n')

In [4]:
env = Texas_Holdem_Simp()
s =  np.random.choice(env.S0)
env.plot(s)

----(1)-----
|P2(3): Q♠ |
|(2): K♥-**|
|P1(5): K♠ |
------------


In [5]:
# a = pi[s]
# print(a)
# env.plot(np.random.choice([s_ for s_ in env.S if env.T(s,a,s_)>0]))

In [6]:
# print(len(env.S0))
# s1 = [s_ for s in env.S0 for s_ in env.S if env.T(s,pi[s],s_)>0]
# print(len(s1))